# Mapeamento de Empresas

O código em questão busca mapear as empresas relacionadas as tags que forem inseridas na busca

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [ ]:
#Armazenando o número de inputs que serão coletados
n_tags = input('Indique o número de tags que você irá buscar: ')

### Tags
Para inserir as tags, lembre-se de que a pesquisa é feita através do Google Search, use isso à seu favor melhorando os resultados de busca através das expressões de busca avançada (basicamente '+', para acrescentar, e '-', para subtrair resultados indesejáveis)

In [ ]:
lista_de_tags = []
for i in range(int(n_tags)):
    tag = input('Insira sua tag, uma por vez: ')
    lista_de_tags.append(tag)
print('Tags {} adicionadas à lista de pesquisa'.format(lista_de_tags))

In [ ]:
#Criando o navegador
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [ ]:
linkedin_link = []

for i in range(len(lista_de_tags)):

    #Abrindo o navegador e realizando a pesquisa
    navegador.get("https://www.google.com.br/")
    navegador.find_element(By.CLASS_NAME, 'gLFyf').click()
    navegador.find_element(By.CLASS_NAME, 'gLFyf').send_keys(lista_de_tags[i],' site:https://www.linkedin.com/company/')
    navegador.find_element(By.CLASS_NAME, 'gLFyf').send_keys(Keys.RETURN)
    
    #Armazenando links
    ##Caso seja desejável mapear até certo número de páginas, criar condição no while e lembrar de iterar a variável
    aux = 1
    while aux:
        elementos_google = navegador.find_elements(By.CLASS_NAME, 'yuRUbf')

        for elemento in elementos_google:
            link = elemento.find_element(By.TAG_NAME, 'a').get_attribute('href')
            linkedin_link.append(link)

    ##Passador de páginas    
        try:
            navegador.find_element(By.XPATH, '//*[@id="pnnext"]/span[2]').click()
        except:
            break

### Login no Linkedin
Para que o código funcione de maneira apropriada, entre no navegador criado, faça o login com sua conta no Linkedin e pressione qualquer tecla no input a seguir

In [ ]:
input('Podemos prosseguir para a coleta de dados? ')

In [ ]:
lista_empresas = []
links_site = []

for i in range(len(linkedin_link)):
    link_aux = linkedin_link[i] + '/about/'
    navegador.get(link_aux)
    try:
        navegador.find_element(By.LINK_TEXT, 'Sobre').click()
    except:
        pass
    
    try:
        nome = navegador.find_element(By.TAG_NAME, 'h1').text
        lista_empresas.append(nome)
    except:
        lista_empresas.append('')
    
    time.sleep(1)
    
    site_aux = navegador.find_element(By.CLASS_NAME, 'overflow-hidden')
    try:
        href_links = site_aux.find_element(By.TAG_NAME, 'a').get_attribute('href')
        links_site.append(href_links)
    except:
        links_site.append('')

### Limpeza de URL's quebradas
Caso exista algum link de uma página excluída do Linkedin, o código acima dará erro.
A página de erro será a que estará no navegador criado.
Para limpá-lo da lista de links copie a linha abaixo em um campo de código
linkedin_link.remove('url da página de erro, menos o "/about/"')

In [ ]:
#Criando DataFrame com os dados coletados
dados = {'Empresa':  lista_empresas,
        'Linkedin': linkedin_link,
        'Site': links_site
        }

df = pd.DataFrame(dados)

In [ ]:
#Visualizando o Dataframe
df

In [ ]:
#Excluindo duplicatas
df2 = df.drop_duplicates(keep='first')

In [ ]:
#Visualizando novo Dataframe
df2

In [ ]:
#Exportando Dataframe em formato xlsx
nome_arquivo = 'Mapeamento de empresas.xlsx'
df2.to_excel(nome_arquivo)